- Source: https://tableau.github.io/server-client-python/docs/api-ref
- "HDC현대산업개발": "2a21240d-3f53-4f02-944d-b01647c3dd51"

In [ ]:
# hr["have_account"] = hr.apply(lambda x:True if x["num"] in all_users["user_name"].tolist() else False, axis=1)

In [36]:
import tableauserverclient as TSC
import pandas as pd
import numpy as np
import pickle as pk

pd.set_option("display.max_rows", None)

In [103]:
hr = pd.read_excel("D:/디지털혁신팀/태블로/직원명단_조직_210630.xlsx")
hr["성명"] = hr["성명"].apply(lambda x:x.replace(" ", ""))
hr = hr[["사번", "EMAIL", "본부명(조직단위기준)", "조직명", "하위그룹", "직무", "직급", "성명"]]
hr = hr.rename({"사번":"num", "EMAIL":"email", "본부명(조직단위기준)":"hq", "조직명":"dep", "하위그룹":"sub", "직무":"job", "직급":"rank", "성명":"name"}, axis=1)
hr = hr.sort_values(by=["job", "rank", "hq", "dep"], axis=0)
hr["num"] = hr["num"].map(str)

hr.loc[hr["num"] == "1", "num"] = "0001"

In [38]:
empls = {str(row["num"]):(row["hq"], row["dep"], row["job"], row["name"]) for _, row in hr.iterrows()}

req_opts = TSC.RequestOptions(pagesize=1000)

## SIgn In

In [3]:
with open("D:/디지털혁신팀/태블로/admin_TOKEN.pkl", "rb") as f:
    token = pk.load(f)
tableau_auth = TSC.PersonalAccessTokenAuth("admin_TOKEN", token)
server = TSC.Server("http://218.153.56.75/", use_server_version=True)

## Create Fundamental Datasets

In [111]:
# all_users.csv
with server.auth.sign_in(tableau_auth):
    all_users, pagination_item = server.users.get(req_options=req_opts)
all_users = [(user.id, user.name, user.fullname, user.site_role) + empls[user.name] if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None) for user in all_users]
all_users = pd.DataFrame(all_users, columns=["user_id", "user_name", "user_fullname", "site_role", "hq", "dep", "job", "name"])

all_users.to_csv("D:/Github/Work/Tableau/all_users.csv", index=False, encoding="euc-kr")

# all_groups.csv
with server.auth.sign_in(tableau_auth):
    all_groups, pagination_item = server.groups.get(req_options=req_opts)
all_groups = [(group.name, group.id) for group in all_groups]
all_groups = pd.DataFrame(all_groups, columns=["group", "group_id"])

all_groups.to_csv("D:/Github/Work/Tableau/all_groups.csv", index=False, encoding="euc-kr")

# user_group.csv
user_group = list()
with server.auth.sign_in(tableau_auth):
    users, pagination_item = server.users.get(req_options=req_opts)
    for user in users:
        pagination_item = server.users.populate_groups(user, req_options=req_opts)
        user_group.extend([(user.id, user.name, user.fullname, user.site_role) + empls[user.name] + (group.name,) if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None, group.name) for group in user.groups if group.name != "All Users"])
user_group = pd.DataFrame(user_group, columns=["user_id", "user_name", "user_fullname", "site_role", "hq", "dep", "job", "name", "group"])

user_group.to_csv("D:/Github/Work/Tableau/user_group.csv", index=False, encoding="euc-kr")

# group_user.csv
group_user = list()
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    for group in groups[1:]:
        pagination_item = server.groups.populate_users(group, req_options=req_opts)
        group_user.extend([(group.name,) + empls[user.name] + (user.site_role,) if user.name in empls.keys() else (group.name, None, None, None, user.name, user.site_role) for user in group.users])

group_user = pd.DataFrame(group_user, columns=["group", "hq", "dep", "job", "name", "site_role"])

group_user.to_csv("D:/Github/Work/Tableau/group_user.csv", index=False, encoding="euc-kr")

## 그룹별 유저 배정

In [71]:
# "HR현황"
ls = hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장")]["num"].tolist()
for num in ls:
    try:
        user_id = all_users.loc[all_users["user_name"] == num, "user_id"].values[0]
    except:
        print(f"No Account; {num}")
#         with server.auth.sign_in(tableau_auth):
#             new_user = TSC.UserItem(num, "Viewer")
#             new_user = server.users.add(new_user)   
#             new_user.fullname = f"{hr.loc[hr["num"] == num, "name"].values[0]}({hr.loc[hr["num"] == num, "dep"].values[0]})"
#             new_user.email = hr.loc[hr["num"] == num, "email"].values[0]
#             new_user = server.users.update(new_user, password=f"hdc{name}")
#             user_id = all_users.loc[all_users["user_name"] == num, "user_id"].values[0]
    with server.auth.sign_in(tableau_auth):
        groups, pagination_item = server.groups.get(req_options=req_opts)
        group = [i for i in groups if i.id == "8bae0a19-c83c-4713-b932-a489754653de"][0]
        server.groups.add_user(group, user_id)

## 그룹별 유저 기준

### 현장 실적 + 예산변경 현황, CS접수현황, HR현황, 협업플랫폼 사용현황
- 사용자 전체

### 분양일보("16065e45-21b3-4e1b-80e1-c7c41079adfd")
```python
hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장") | ((hr["dep"] == "도시개발1팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "도시개발2팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "도시정비1팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "도시정비2팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "마케팅팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "민간수주1팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "민간수주2팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "고객서비스팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "경영분석팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "재무팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "회계팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "디지털플랫폼팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "미래전략팀") & (hr["job"] == "팀장")) | ((hr["dep"] == "정보기술팀") & (hr["job"] == "팀장")) | (hr["num"] == "6110") | (hr["num"] == "4001380")]
```
- 회장님 비서?, 상임자문?, 부문장?, 김대중?

### 수주정보
```python
hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장")]
```
- 회장님 비서?, 상임자문?

### 본부장 회의("01b579e9-e0b0-458d-b12a-94292171d3a7")
```python
hr[(hr["job"] == "회장") | (hr["job"] == "부회장") | (hr["job"] == "대표이사사장") | (hr["job"] == "본부장") | (hr["job"] == "팀장") | (hr["job"] == "실장") | (hr["job"] == "그룹장") | (hr["job"] == "부문장") | (hr["job"] == "단장") | (hr["job"] == "리더") | (hr["job"] == "PM") | (hr["job"] == "건축PM") | (hr["num"] == "6447") | (hr["num"] == "6406") | (hr["num"] == "6486") | (hr["num"] == "6427") | (hr["num"] == "6014") | (hr["num"] == "5704") | (hr["num"] == "5720") | (hr["num"] == "5882") | (hr["num"] == "4001380") | (hr["num"] == "4100435") | (hr["num"] == "6281") | (hr["num"] == "6363") | (hr["num"] == "5899") | (hr["num"] == "6048")]
```
- 회장님 비서?, 상임자문?, 김주완?, 성기창?, 오재환?

## Remove user

In [26]:
user_id = "371ace7c-8abb-484e-9c61-ac1ab2247f4d"
with server.auth.sign_in(tableau_auth):
    server.users.remove(user_id)

## Modify User Information

In [32]:
# for _, row in all_users.iterrows():
#     if row["dep"] != None:
#         print(row["user_id"], row["user_fullname"], row["name"], row["dep"])

2c2ceaa8-9387-4edb-8ab2-38b17f686304 김동진 김동진 건축PM
c232b3be-3aac-4f4f-83fb-bc8a15dab963 배영선 배영선 건축PM
e17666e4-a736-40a7-badc-865a60ebd7f2 서태철 서태철 광주학동4구역
cf30b80d-8e3a-4128-995a-acbe79a7090f 이훈석 이훈석 건설기획팀(신규준비)
f0a544b2-1f42-4f3d-b20f-bcd428b11d6a 김행대 김행대 서울숲IPARK리버포레
123c4e57-2fc6-4a62-8837-d3c1277ee1c6 정상민 정상민 외주구매팀
af48c1c7-5d45-4e6a-bd52-e028d35013cf 김영도 김영도 현대엘리베이터충주신공장
55a1b5a0-fe20-4711-85d4-514c0d1aaec4 김유겸 김유겸 고속국도제32호선아산-천안간건설공사(4공구)
06f0c545-3f70-47ac-a6e3-0fd195c4ea68 이문경 이문경 부산동래래미안아이파크
15d539bf-721c-42f7-a3bb-a0b6cecbf3eb 이종일 이종일 TheH Firstier IPARK(개포1 주택재건축)
ab1b3edd-f03c-4358-baf2-695f7bab45f5 고영진 고영진 고속국도제32호선아산-천안간건설공사(5공구)
102a7f7f-6fa3-41b2-ab18-8fb2357fa3d5 이창규 이창규 건축PM
7f57aeae-354b-4ed7-acf0-fcb7a29ef108 김달운 김달운 아시아드레이카운티(부산거제2구역)
2bc4b236-5eb8-4b69-a32e-5ef51a167d5a 한희태 한희태 김포풍무2지구도시개발사업
9d798eca-8cfe-461f-9a23-5e654cc799c7 김한덕 김한덕 수원센트럴아이파크자이(수원팔달10구역재개발)
4b68efef-e3cc-4820-af1e-54e23bcd5cda 최영준 최영준 아현2구역재건축
693792b3-5945-468a-8652-22479d429b45 김창환 김창환 용산병원부지개발

In [ ]:
# with server.auth.sign_in(tableau_auth):
#     user = server.users.get_by_id(row["user_id"])
#     user.fullname = f"{row['name']}({row['dep']})"
    
#     user_rev = server.users.update(user)

## have_account

In [ ]:
hr["account"] = hr.apply(lambda x:True if x["num"] in all_users["user_name"].tolist() else False, axis=1)

In [27]:
### user.remove()

user_id = "371ace7c-8abb-484e-9c61-ac1ab2247f4d"
with server.auth.sign_in(tableau_auth):
    server.users.remove(user_id)
    hr["account"] = hr.apply(lambda x:True if x["num"] in all_users["user_name"].tolist() else False, axis=1)

In [33]:
hr.to_csv("D:/디지털혁신팀/태블로/hr.csv", encoding="euc-kr")

## Projects

In [72]:
# with server.auth.sign_in(tableau_auth):
#     all_project_items, pagination_item = server.projects.get(req_options=req_opts)
#     print([(prj.name, prj.id) for prj in all_project_items])

## Workbooks

In [307]:
# with server.auth.sign_in(tableau_auth):
#     all_wbs, pagination_item = server.workbooks.get()
# all_wbs = [(wb.project_name, wb.name) for wb in all_wbs]
# all_wbs = pd.DataFrame(all_wbs, columns=["prj", "wb"])
# all_wbs = all_wbs.sort_values(by=["prj", "wb"])

## Groups

## groups.get(), groups.populate_users()

In [77]:
with server.auth.sign_in(tableau_auth):
    all_groups, pagination_item = server.groups.get(req_options=req_opts)
all_groups = [(group.name, group.id) for group in all_groups]
all_groups = pd.DataFrame(all_groups, columns=["group", "group_id"])

# all_groups.to_csv("D:/Github/Work/Tableau/all_groups.csv", index=False, encoding="euc-kr")

In [24]:
user_name = "creator1"
site_role = "Creator"
newU = TSC.UserItem(user_name, site_role)

print(newU.name, newU.site_role, newU.workbooks)

UnpopulatedPropertyError: User item must be populated with workbooks first.

## Add Users to Groups

In [73]:
# with server.auth.sign_in(tableau_auth):
#     groups, pagination_item = server.groups.get(req_options=req_opts)
#     users, pagination_item = server.users.get(req_options=req_opts)
#     for _, row in hr[hr["check"].notna()].iterrows():
#         idx = all_groups[all_groups["group"] == row["check"]].index[0]
#         group = groups[idx]
        
#         user_idx = all_users[all_users["user_name"] == row["num"]].index[0]
#         user_id = users[user_idx].id
    
#         server.groups.add_user(group, user_id)

In [76]:
keyword = "호미"
num = "5634"

group_idx = all_groups[all_groups["group"].str.contains(keyword)].index[0]
with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    group = groups[group_idx]
    
    user_idx = all_users[all_users["user_name"] == num].index[0]
    users, pagination_item = server.users.get(req_options=req_opts)
    user_id = users[user_idx].id
    
    server.groups.add_user(group, user_id)

### users.populate_workbooks()

In [96]:
# with server.auth.sign_in(tableau_auth):
#     user = server.users.get_by_id("515c7d10-953c-4bfc-af9a-e87cf34ccb85")
#     page_n = server.users.populate_workbooks(user)
#     for workbook in user.workbooks :
#         if workbook.project_name == "HDC현대산업개발":
#             print(workbook.name)

## Create Accounts

In [115]:
with server.auth.sign_in(tableau_auth):
    name = "1101296"
    site_role = "Viewer"
    new_user = TSC.UserItem(name, site_role)
    
    new_user = server.users.add(new_user)   
    new_user.fullname = "황승환(디지털플랫폼팀)"
    new_user.email = "hwangsh@hdc-dvp.com"
    
    new_user = server.users.update(new_user, password=f"hdc{name}")